<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag-api.html)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc..

In [ ]:
import dask.bag as db

# each element is an integer
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
b

In [ ]:
import os

# each element is a text file of JSON lines
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b

In [ ]:
b.npartitions

In [ ]:
b.visualize()

## Manipulation

`Bag` objects implement the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

As with `Array` and `DataFrame` objects, operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution.

In [ ]:
def iseven(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

c = (b.filter(iseven)           # Filter out odd elements
      .map(lambda x: x ** 2))   # Square all elements

c

In [ ]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

## Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory. This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [ ]:
filenames = os.path.join('data', 'accounts.*.json.gz')

# Read the accounts data in as a bag. Each line is an element
lines = db.read_text(filenames)

# Look at the first element
lines.take(1)

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [ ]:
import json
js = lines.map(json.loads)

js.take(1)

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [ ]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

In [ ]:
def count_transactions(d):
    return {'name': d['name'],
            'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

In [ ]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

In [ ]:
# Total number of transactions for all of the Alice's
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .sum()
   .compute())

### Exercise: Compute all the unique names

Hint: you'll need to use [`Bag.pluck`](http://dask.pydata.org/en/latest/bag-api.html#dask.bag.Bag.pluck)
and [`Bag.distinct`](http://dask.pydata.org/en/latest/bag-api.html#dask.bag.Bag.distinct).

<button data-toggle="collapse" data-target="#sol1" class='btn btn-primary'>Solution</button>
<div id="sol1" class="collapse">
```python
js.pluck('name').distinct().compute()
```

## Use `flatten` to de-nest

In the example below we see the use of `flatten` to flatten results.  We compute the average amount for all transactions for all Alices.

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')   # Just the alices
   .pluck('transactions')                              # Pull out the 'transactions' field
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')  # Just the alices
   .pluck('transactions')                             # Pull out the 'transactions' field
   .flatten()                                         # Flatten the nested lists
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')  # Just the alices
   .pluck('transactions')                             # Pull out the 'transactions' field
   .flatten()                                         # Flatten the nested lists
   .pluck('amount')                                   # Pull out the 'amount' field
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')  # Just the alices
   .pluck('transactions')                             # Pull out the 'transactions' field
   .flatten()                                         # Flatten the nested lists
   .pluck('amount')                                   # Pull out the 'amount' field
   .mean()                                            # Take the mean of all amounts
   .compute())

### Exercise: How many transactions did all the Alices have?

<button data-toggle="collapse" data-target="#sol2" class='btn btn-primary'>Solution</button>
<div id="sol2" class="collapse">
```python
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .count()
   .compute())
```

## Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
# names grouped by length
b.groupby(len).compute()

In [ ]:
b = db.from_sequence(list(range(10)))
# numbers grouped by even/odd
b.groupby(lambda x: x % 2).compute()

In [ ]:
# Get max for each group
(b.groupby(lambda x: x % 2)
  .starmap(lambda k, v: (k, max(v)))
  .compute())

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [ ]:
(b.foldby(lambda x: x % 2,
          binop=lambda acc, x: acc if acc > x else x,
          combine=lambda acc1, acc2: acc1 if acc1 > acc2 else acc2)
  .compute())

### Example with account data

We find the number of people with the same name.

In [ ]:
%%time

(js.groupby(lambda item: item['name'])
   .starmap(lambda k, v: (k, len(v)))
   .compute())

In [ ]:
%%time

# This one is comparatively fast and produces the same result.
(js.foldby(key='name',
           binop=lambda total, x: total + 1, 
           initial=0, 
           combine=lambda a, b: a + b, 
           combine_initial=0)
   .compute())

### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

<button data-toggle="collapse" data-target="#sol3" class='btn btn-primary'>Solution</button>
<div id="sol3" class="collapse">
```python
def sum_amounts(d):
    transactions = d['transactions']
    return sum(t['amount'] for t in transactions)

(js.foldby(key='name',
           binop=lambda total, x: total + sum_amounts(x), 
           initial=0, 
           combine=lambda a, b: a + b, 
           combine_initial=0)
   .compute())
```

## DataFrames

Pandas is faster than Pure Python.  In the same way, `dask.dataframe` can be faster than `dask.bag`.  We will work more with dataframes later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion - once the data can be made into a data-frame, then complex split-spply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [ ]:
df = js.to_dataframe()
df.head()

This is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..  

In [ ]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'],
             'name': record['name'],
             'amount': transaction['amount'],
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

In [ ]:
df = transactions.to_dataframe()
df.head()

This now looks like a well-defined data-frame, and we can apply Pandas-like computations to it efficiently.

In [ ]:
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.